In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
from torch.nn import Parameter
from torch import Tensor
import torch.nn.functional as F
import math
import sys
import os 

In [2]:
current_dir = os.getcwd()
path = current_dir[: current_dir.find("Cinnamon2023-HaNoiTeam")]
sys.path.append(f'{path}Cinnamon2023-HaNoiTeam/experiments/Tammy/models')
from LSTM_Tammy import LSTM 

In [3]:
'''
STEP 1: LOADING DATASET
'''
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)
 
test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())
 
batch_size = 100
n_iters = 6000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

In [4]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
 
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [5]:
input_dim = 28
hidden_dim = 128
layer_dim = 1  # ONLY CHANGE IS HERE FROM ONE LAYER TO TWO LAYER
output_dim = 10
 
model = LSTM(input_dim, hidden_dim, output_dim, layer_dim)
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()
 
'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1
 
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [6]:
seq_dim = 28 

loss_list = []
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.view(-1, seq_dim, input_dim))
        labels = Variable(labels)
          
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)

        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
        
        loss_list.append(loss.item())
        iter += 1
         
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                images = Variable(images.view(-1 , seq_dim, input_dim))
                outputs = model(images)
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum()
        
            accuracy = 100 * correct / total
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 1.1953420639038086. Accuracy: 70.44999694824219
Iteration: 1000. Loss: 0.27701228857040405. Accuracy: 91.31999969482422
Iteration: 1500. Loss: 0.13084924221038818. Accuracy: 94.01000213623047
Iteration: 2000. Loss: 0.16885727643966675. Accuracy: 95.38999938964844
Iteration: 2500. Loss: 0.09800630807876587. Accuracy: 96.25
Iteration: 3000. Loss: 0.12472598999738693. Accuracy: 97.0999984741211
Iteration: 3500. Loss: 0.1098584458231926. Accuracy: 97.04000091552734
Iteration: 4000. Loss: 0.08067778497934341. Accuracy: 97.20999908447266
Iteration: 4500. Loss: 0.0362263098359108. Accuracy: 97.83000183105469
Iteration: 5000. Loss: 0.02314344234764576. Accuracy: 97.87999725341797
Iteration: 5500. Loss: 0.011920955032110214. Accuracy: 97.91999816894531
Iteration: 6000. Loss: 0.013869612477719784. Accuracy: 97.87000274658203
